## Genetic Algorithm

in this project we are going to see how a big problem with lots of data can be solved by genetic algorithm, and see how answers(known as chromosomes) are getting optimized by time, and see and analyse mutate and crossover methods.

In [25]:
import pandas as pd
import numpy as np
import random
from math import ceil

- A gene is the percentage of a stock, so chromosome is a series of total number of genes(400 in this case)
- sum of genes of a chromosome must be 1
- to produce primary population, we use 400 random genes and then normalize them to satisfy chromosome condition.
- according to problem we must have risk < 60% and return > 1000% so I give a bigger negative ratio to risk and a 1 ration to return

In [26]:
def fitness(col):
    return (col * (stocks['return'] - 5 * stocks['risk'])).sum()

def sreturn(col):
    return (col * stocks['return']).sum()
    
def srisk(col):
    return (col * stocks['risk']).sum()

Q1: I found a good balance of time and accuracy in population = 300, large population takes much time to generate a new generation while small population causes sameness of data and does not produce diverse children so we may not have good chromosome.

Q2: increasing popultion in each generation makes program slower and it push data to latests generation, it can be good or bad, after some generations we have less diverse data so increasig population does not really help as we get close to last generations.

I used dataframe to keep genes, rows are people and colomns are genes

In [27]:
stocks = pd.read_csv("sample.csv")
n_stocks = stocks.shape[0]

population = 300
half = int(population / 2)
ranks = list(range(1,population + 1))
people = pd.DataFrame(np.random.randint(0,100,size=(n_stocks, population)))


people = people.apply(lambda x : x/x.sum(), axis=0)

people.loc['fitness'] = people.apply(fitness,axis=0)
print(people.loc['fitness'].max())
people.sort_values(by = 'fitness', inplace=True, axis=1)
people.drop('fitness',axis=0, inplace=True)
people.columns = range(people.columns.size)

-1.392222623251269


Q3:
- crossover makes data more diverse so using only mutation causes sameness and does not produce good data. 
- mutation is good for sudden progress and prevent pushing data to the best fitting chromosomes, using only crossover makes progress much slower because a good mutate can make a good chromosome and that chromosome reproduce and get us closer to answer and a bad one does not affect whole data and will be removed naturally in next generations.

I reduce mutation rate every generation because when chromosomes are getting close to answer, we already have good genes and mutating them may causes outlier genes.

In [28]:
mutation_rate = -0.05
mutation_init = int(0.07 * population)

I use uniform crossover and rank based selection of parents for next generation.

Note:
- commented code is a alternative way of crossover, 1-point crossover

Q4: I tried cutting off minimum genes in every generation and spread their weight to other genes, it has some pros and cons, it can help to get high fitting answers in early generations but in long time by denying these genes, we can not have diverse chromosomes.


In [29]:

for cycle in range(100):
    
    parents = random.choices(ranks,weights=ranks,k=population)
    
    for i in range(int(population/2)):
        child1 = random.choices([True,False] , k = n_stocks)
        child2 = np.invert(child1)
        people[f'new{i}'] = people[parents[i] - 1] * child1 + people[parents[i + half] - 1] * child2
        people[f'new{i+half}'] = people[parents[i] - 1] * child2 + people[parents[i + half] - 1] * child1 

        # crossover = random.randint(0,n_stocks)
        # child1 = [False] * crossover + [True] * (n_stocks - crossover)
        # child2 = np.invert(child1)
        # people.loc[len(people)] = people.iloc[parents[i] - 1] * child1 + people.iloc[parents[i + half] - 1] * child2
        # people.loc[len(people)] = people.iloc[parents[i] - 1] * child2 + people.iloc[parents[i + half] - 1] * child1 


    people = people.iloc[:,population:]
    

    for mutate in range(ceil(mutation_init)):
        person = random.randint(0, population - 1)
        gen = random.randint(0, n_stocks - 1)
        people.iloc[gen,person] = random.randint(0,1)

    mutation_init += mutation_rate

    people = people.apply(lambda x : x/x.sum(), axis=0)
    people.loc['fitness'] = people.apply(fitness,axis=0)
    people.sort_values(by = 'fitness', inplace=True, axis= 1)
    print(cycle, people.loc['fitness'].max())
    people.drop('fitness',axis=0, inplace=True)
    people.columns = range(people.columns.size)


C:\Users\asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if __name__ == "__main__":
C:\Users\asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


0 -1.3733105958031198
1 -0.8754568557665854
2 -1.0383563593731004
3 0.18521641326793592
4 -0.20236171193620672
5 -0.2816543369996051
6 -0.14577228761198857
7 -0.19908784562450776
8 -0.2545509503598539
9 1.101527907297298
10 0.5451476310250488
11 0.6108905352181488
12 0.6704543846676454
13 0.7015613933845282
14 0.7901362380825364
15 0.9538844776467181
16 0.7825752887409094
17 0.8392805269906225
18 1.8012629181225102
19 1.4817742306796684
20 1.3128548039538228
21 1.41626505993705
22 1.3773713356186041
23 3.8780206437754154
24 3.1025577956007924
25 2.965302229559329
26 2.6453005676188353
27 2.195849038873099
28 2.1815269980045326
29 2.228000763448307
30 2.25465564378989
31 2.1570616292548195
32 2.130846359342878
33 2.185486117055969
34 2.1703661427349275
35 2.2034375495525937
36 2.2887956121914748
37 2.2849706028846297
38 2.3711716585274054
39 2.4832629740633356
40 2.4713995845597063
41 2.5014665427259732
42 2.6250325446139566
43 2.6780181457692063
44 2.922404877272185
45 3.01849423588681

As we can every generation we have a good progress and chromosomes are getting higher fitnesses

in the end I decide to cut off minmum genes and spread their weight to others

In [30]:
minimum_stocks = 35

people = people.apply(lambda x: x * (x > x.quantile(1 - minimum_stocks/n_stocks)), axis = 0)
people = people.apply(lambda x : x/x.sum(), axis=0)

In [31]:
people.loc['return'] = people.apply(sreturn,axis=0)
people.loc['risk'] = people.apply(srisk,axis=0)

print(people.loc['return'].max())

12.655714805746772


results is saved in 'res.csv' file, we can see we have a good 1200% return with less than 50% risk

In [32]:
people.to_csv("res.csv")


Q5: small population may causes that, or a very good generation can causes sameness, to prevent that I used uniform crossover and rank-based selection. having a good chromosome and selecting by amount of fitness push data to that chromosome and we have several good answer but not best ones.

Q6: limiting cycles can be a good way to end a program which chromosomes can not meet the condition of problem, tracking progress rate also can help us, slow progress means we reached out highest as we can and making new and more fitting chromosome is hard.